### Introduction

The rapid growth of urbanization and industrial production has significantly impacted the environment, leading to an alarming increase in waste generation. Managing this waste effectively has become a global priority. **Recycling**, a key solution to this challenge, transforms waste into valuable resources through proper sorting and processing. This project focuses on enabling automated systems, such as smart waste-sorting robots and intelligent waste bins, to classify waste accurately by analyzing trash images.

This tutorial outlines the step-by-step process of training the lightweight Convolutional Neural Network (CNN) **EfficientNet-B0** on a custom dataset of 2,527 images representing different types of trash. The resulting model serves as a foundation for lightweight waste management image classification systems, designed for real-world applications such as live deployment in smart bins.

---

#### About EfficientNet

**EfficientNet** is a family of models (B0–B7) that balance computational efficiency and accuracy. Higher-numbered models require more computational resources and larger datasets, while lower-numbered models are lightweight and suitable for resource-constrained environments. 

For this project, we selected the **EfficientNet-B0** CNN model due to its efficiency, ease of training, and minimal computational demands. This lightweight model is ideal for deployment in distributed systems, such as smart bins, where compactness and low-power requirements are critical.

---

#### About Our Data

Our dataset is imported from the Kaggle directory of **Gary Huang and Mindy Yang's** TrashNet repository (FeyZazkefe on Kaggle). It contains 2,527 images of trash, distributed across six categories:

1. **Cardboard**: 403 images
2. **Glass**: 501 images
3. **Metal**: 410 images
4. **Paper**: 594 images
5. **Plastic**: 482 images
6. **Trash**: 137 images

#### Step 1: Import Training Image Dataset & Initiate Code Carbon

In [3]:
from codecarbon import EmissionsTracker

# Start CodeCarbon tracking
tracker = EmissionsTracker(allow_multiple_runs=True)
tracker.start()

"""
import kagglehub

# Downloaded TrashNet dataset from Kaggle: https://www.kaggle.com/datasets/feyzazkefe/trashnet
# Dataset by feyzazkefe, originally sourced from Stanford TrashNet.

target_path = "./data"
path = kagglehub.dataset_download("feyzazkefe/trashnet", path=target_path)
print("Path to dataset files:", path)
"""

[codecarbon WARNING @ 00:05:08] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 00:05:08] [setup] RAM Tracking...
[codecarbon INFO @ 00:05:08] [setup] GPU Tracking...
[codecarbon INFO @ 00:05:08] No GPU found.
[codecarbon INFO @ 00:05:08] [setup] CPU Tracking...
[codecarbon WARNING @ 00:05:08] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 00:05:09] We saw that you have a AMD Ryzen 5 7520U with Radeon Graphics but we don't know it. Please contact us.
[codecarbon INFO @ 00:05:09] CPU Model on constant consumption mode: AMD Ryzen 5 7520U with Radeon Graphics
[codecarbon INFO @ 00:05:09] >>> Tracker's metadata:
[codecarbon INFO @ 00:05:09]   Platform system: Linux-6.8.0-49-generic-x86_64-with-glibc2.39
[codecarbon INFO @ 00:05:09]   Python version: 3.10.15
[codecarbon INFO @ 00:05:09]   CodeCarbon version: 2.8.

'\nimport kagglehub\n\n# Downloaded TrashNet dataset from Kaggle: https://www.kaggle.com/datasets/feyzazkefe/trashnet\n# Dataset by feyzazkefe, originally sourced from Stanford TrashNet.\n\ntarget_path = "./data"\npath = kagglehub.dataset_download("feyzazkefe/trashnet", path=target_path)\nprint("Path to dataset files:", path)\n'

[codecarbon INFO @ 00:05:27] Energy consumed for RAM : 0.000023 kWh. RAM Power : 5.572976589202881 W
[codecarbon INFO @ 00:05:27] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 00:05:27] 0.000200 kWh of electricity used since the beginning.


#### Step 2: Import packages

In [5]:
import numpy as np
from PIL import Image
import os
import shutil

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, classification_report

#### Step 3: Split the Imported Image Data into Training, Validation, and Test Sets

After replacing **`original_data_dir`** with the correct directory, running the code below will produce the following splits of the dataset:

1. **Training Images**: 1815 images used to train the model.
2. **Test Images**: 508 images reserved for evaluating the model’s performance after training.
3. **Validation Images**: 204 images used to tune the model during training and prevent overfitting.

This split ensures a balanced dataset division, optimizing the model’s training and evaluation process.

In [6]:
# Paths
original_data_dir = "data/trashnet" #replace with wherever the original image data folder is
split_data_dir = "data/split_data"
categories = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]

# Create directories for train, val, and test splits
for split in ["train", "val", "test"]:
    for category in categories:
        os.makedirs(os.path.join(split_data_dir, split, category), exist_ok=True)

# Split data for each category
for category in categories:
    category_path = os.path.join(original_data_dir, category)
    images = os.listdir(category_path)
    images = [img for img in images if img.endswith(('.jpg', '.png'))]  # Filter image files

    # Split into train+val and test (80-20)
    train_val, test = train_test_split(images, test_size=0.2, random_state=42)
    
    # Further split train+val into train and val (90-10 of train+val)
    train, val = train_test_split(train_val, test_size=0.1, random_state=42)

    # Copy files to split_data directory
    for split, split_images in zip(["train", "val", "test"], [train, val, test]):
        for img in split_images:
            src_path = os.path.join(category_path, img)
            dest_path = os.path.join(split_data_dir, split, category, img)
            shutil.copy(src_path, dest_path)

#### Step 4: Define Transformation Functions for Training and Testing Image Sets

To prepare our images for input into the EfficientNet-B0 model, we define specific transformation pipelines for both the training and testing datasets. These transformations ensure that the images are in the correct format, augmented for diversity, and normalized for compatibility with the pre-trained model.

- **Standardization**: EfficientNet-B0 requires input images with dimensions of **(224, 224)** (height × width). This resizing is applied uniformly to both the training and testing datasets.
- **Data Augmentation (Training Only)**: To enhance the diversity of the training data, random augmentations are applied:
  - **Random Horizontal Flips**: Introduce variability by flipping images horizontally with a certain probability.
  - **Random Rotations**: Images are randomly rotated up to ±10 degrees to simulate different orientations.
- **Normalization**: Both the training and testing datasets are normalized to align with the ImageNet normalization method used during EfficientNet's pre-training:
  - **Mean**: [0.485, 0.456, 0.406] (per channel for RGB images).
  - **Standard Deviation**: [0.229, 0.224, 0.225].

These transformations ensure that the training data is varied and robust while maintaining consistency for the test data.


In [7]:
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to EfficientNet input size
    transforms.RandomHorizontalFlip(),  # Lightweight and effective
    transforms.RandomRotation(10),  # Augment slightly with small angles
    transforms.ToTensor(),  # Convert image to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to EfficientNet input size
    transforms.ToTensor(),  # Convert image to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

In [8]:
# Load datasets
train_dataset = ImageFolder(root="data/split_data/train", transform=transform_train)
val_dataset = ImageFolder(root="data/split_data/val", transform=transform_test)
test_dataset = ImageFolder(root="data/split_data/test", transform=transform_test)

# Define data loaders & batch size
batch_size = 32 # Can also be 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

#### Step 5: Load the EfficientNet-B0 Model, Define the Number of Output Classes, and Enable GPU Usage (if Available)

First, we load the EfficientNet-B0 model with **pre-trained weights**. Pre-trained weights refer to the parameters of a neural network that have already been trained on a large dataset (e.g., ImageNet). This allows us to leverage a model that has learned general image features, such as edges, textures, and shapes, without needing to train it from scratch.

- **Initialize Pretrained EfficientNet-B0**: The model is initialized with these pre-learned parameters.
- **Set Output Classes**: We specify the number of output classes for our classification task. In this case, the dataset contains **6 classes** (e.g., cardboard, glass, metal, paper, plastic, trash).
- **Enable GPU Usage**: The code checks if a GPU is available. If so, the model is moved to the GPU to take advantage of faster computations during training and evaluation.

This step ensures the model is ready for training with the appropriate number of output classes and optimized for available hardware.

In [9]:
# Load in efficientnet_b0
weights = EfficientNet_B0_Weights.IMAGENET1K_V1
model = efficientnet_b0(weights=weights)

# Define class number
num_classes = 6
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

# Print class names from training data
print("Classes:", train_dataset.classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


[codecarbon INFO @ 00:06:12] Energy consumed for RAM : 0.000093 kWh. RAM Power : 5.572976589202881 W
[codecarbon INFO @ 00:06:12] Energy consumed for all CPUs : 0.000708 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 00:06:12] 0.000801 kWh of electricity used since the beginning.


### Step 6: Define the Loss Function, Optimizer, and Training / Validation / Testing Pipeline

In this step, we define the loss function, optimizer, and a robust pipeline for training, validating, and testing the model. These components are critical for optimizing the model’s performance and ensuring its ability to generalize to unseen data.

---

### **Key Components**

- **Class Weights**: To address class imbalances in the dataset, class weights are computed using the distribution of training samples. These weights ensure that underrepresented classes are given more importance during training by penalizing their misclassification.
- **Loss Function**: The `nn.CrossEntropyLoss` function, combined with class weights, is used to handle multi-class classification and mitigate the effects of class imbalance.
- **Optimizer**: The Adam optimizer is chosen for its efficiency and ability to handle sparse gradients. A learning rate of `0.0001` is used to enable stable and precise updates.

---

### **Training Pipeline**

- **`train_model_with_early_stopping` Function**:
  - Trains the model for a specified number of epochs (default: 20) while incorporating early stopping to prevent overfitting.
  - Early stopping monitors validation loss and halts training if there is no improvement for a defined patience period (default: 3 epochs).
  - The best model (with the lowest validation loss) is saved to the `Model_save` directory.
  - During each epoch:
    1. The model processes the training data in batches, computes the loss, and updates weights via backpropagation.
    2. Training loss and accuracy are calculated and printed for every epoch.
    3. Validation is performed after each epoch using the `validate_model_with_loss` function.

- **`validate_model_with_loss` Function**:
  - This function evaluates the model on the validation dataset in evaluation mode (`model.eval()`), ensuring layers like dropout and batch normalization behave correctly.
  - Calculates validation loss and accuracy to monitor performance across epochs.
  - The validation loss is used for the early stopping mechanism.

---

### **Testing with Detailed Metrics**

- **`validate_model_with_metrics_and_report` Function**:
  - After training, the model is evaluated on the test dataset to assess its generalization ability. This function computes detailed performance metrics, including:
    - **Precision, Recall, and F1 Score**: Measures how well the model identifies each class and balances false positives and false negatives.
    - **AUC (Area Under Curve)**: Indicates the model's ability to distinguish between classes, calculated using a one-vs-rest approach.
  - Produces a **classification report** summarizing the performance of the model for each class.
  - Outputs metrics such as precision, recall, F1 score, and AUC, offering a comprehensive evaluation of the model.

---

### **Improvements in the Pipeline**

1. **Handles Class Imbalances**: Class weights are calculated dynamically based on training data distribution, ensuring balanced learning.
2. **Early Stopping**: Prevents overfitting by halting training when the validation loss stops improving.
3. **Detailed Testing Metrics**: Incorporates precision, recall, F1 score, and AUC to provide a deeper understanding of the model’s performance beyond accuracy.
4. **Model Checkpointing**: Automatically saves the best model (based on validation loss) for final evaluation and deployment.

---

In [10]:
# Define loss function with class weights and optimizer
class_counts = Counter(train_dataset.targets)
class_weights = torch.tensor([1.0 / count for count in class_counts.values()], device=device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

def train_model_with_early_stopping(model, train_loader, val_loader, criterion, optimizer, epochs=20, patience=3):
    # Create directory for saving the model if it doesn't exist
    model_dir = 'Model_save'
    model_path = os.path.join(model_dir, 'Efficientnet_trash_classifier_final.pth')

    best_val_loss = float('inf')  # Initialize the best validation loss as infinity
    patience_counter = 0  # Count epochs with no improvement

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

        # Validation step
        val_loss, val_acc = validate_model_with_loss(model, val_loader, criterion)

        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

        # Early stopping logic
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), model_path)  # Save the best model
            print(f"Validation loss improved. Saving the model to {model_path}.")
        else:
            patience_counter += 1
            print(f"No improvement in validation loss. Patience counter: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print("Early stopping triggered. Stopping training.")
            break

    # Load the best model
    print(f"Loading the best model from {model_path}.")
    model.load_state_dict(torch.load(model_path))
    return model

# Helper function to validate the model and calculate loss
def validate_model_with_loss(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    val_loss = running_loss / len(val_loader)
    val_acc = correct / total
    return val_loss, val_acc

# Evaluate on the test set
model = efficientnet_b0(num_classes=6)  # Adjust num_classes as per your task
model.load_state_dict(torch.load('Model_save/Efficientnet_trash_classifier_final.pth'))  # Load the weights
model.to(device)  # Move to the appropriate device (CPU/GPU)
model.eval()  # Set to evaluation mode

# Updated validation function to include classification report
def validate_model_with_metrics_and_report(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            probs = torch.nn.functional.softmax(outputs, dim=1)  # Convert logits to probabilities
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    # F1 Score, Precision, Recall
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    # AUC (one-vs-rest)
    auc = roc_auc_score(all_labels, all_probs, multi_class='ovr')

    # Classification report
    class_report = classification_report(all_labels, all_preds, target_names=test_loader.dataset.classes)
    print("\nClassification Report:")
    print(class_report)

    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC: {auc:.4f}")

    return precision, recall, f1, auc, class_report

/tmp/ipykernel_162476/2146259924.py:89: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('Model_save/Efficientnet_trash_classifier_final.pth'))

[codecarbon INFO @ 00:06:27] Energy consumed for RAM : 0.000116 kWh. RAM Power : 5.572976589202881 W
[codecarbon INFO @ 00:06:27] Energy consumed for all CPUs : 0.000885 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 00:06:27] 0.001001 kWh of electricity used since the beginning.


In [49]:
"""
# Function to train our model
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total

        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

        # Validate after each epoch
        validate_model(model, val_loader)

# Function to validate our model
def validate_model(model, val_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Generate a classification report
    print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))
    """

'\n# Function to train our model\ndef train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):\n    model.train()\n    for epoch in range(epochs):\n        running_loss = 0.0\n        correct = 0\n        total = 0\n\n        for inputs, labels in train_loader:\n            inputs, labels = inputs.to(device), labels.to(device)\n\n            optimizer.zero_grad()  # Zero the parameter gradients\n            outputs = model(inputs)  # Forward pass\n            loss = criterion(outputs, labels)  # Calculate loss\n            loss.backward()  # Backward pass\n            optimizer.step()  # Update weights\n\n            # Statistics\n            running_loss += loss.item()\n            _, predicted = torch.max(outputs, 1)\n            correct += (predicted == labels).sum().item()\n            total += labels.size(0)\n\n        epoch_loss = running_loss / len(train_loader)\n        epoch_acc = correct / total\n\n        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_l

#### Step 7: Train the Model to Obtain Trained Weights

In this step, we define the model training parameters and execute the training process to optimize the model's performance.

- **Epochs**: Set to **20**, allowing the model to iterate up to 20 times over the dataset to minimize the loss.
- **Patience**: Set to **3**, enabling the model to continue training for a maximum of 3 consecutive epochs without improvement in validation loss before stopping early.

During training:
- For each epoch, a report is generated showing the **training loss**, **training accuracy**, **validation loss**, and **validation accuracy**.
- If the validation loss improves, the model's weights are saved.

After training:
- The model automatically loads the weights corresponding to the best validation performance.

This ensures that the training process is efficient, prevents overfitting, and uses early stopping to identify the optimal set of weights.


In [50]:
# Train the model
# train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)
model = train_model_with_early_stopping(model, train_loader, val_loader, criterion, optimizer, epochs=20, patience=3)

Epoch 1/20, Loss: 1.4404, Accuracy: 0.5477
Validation Loss: 0.9490, Validation Accuracy: 0.7794
Validation loss improved. Saving the model to Model_save/Efficientnet_trash_classifier_final.pth.
Epoch 2/20, Loss: 0.7612, Accuracy: 0.8006
Validation Loss: 0.5038, Validation Accuracy: 0.8578
Validation loss improved. Saving the model to Model_save/Efficientnet_trash_classifier_final.pth.
Epoch 3/20, Loss: 0.4247, Accuracy: 0.8749
Validation Loss: 0.3457, Validation Accuracy: 0.8824
Validation loss improved. Saving the model to Model_save/Efficientnet_trash_classifier_final.pth.
Epoch 4/20, Loss: 0.2769, Accuracy: 0.9124
Validation Loss: 0.2822, Validation Accuracy: 0.9167
Validation loss improved. Saving the model to Model_save/Efficientnet_trash_classifier_final.pth.
Epoch 5/20, Loss: 0.1998, Accuracy: 0.9466
Validation Loss: 0.2145, Validation Accuracy: 0.9363
Validation loss improved. Saving the model to Model_save/Efficientnet_trash_classifier_final.pth.
Epoch 6/20, Loss: 0.1332, Accu

/tmp/ipykernel_148121/1995310198.py:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


#### Step 8: Interpreting the Model's Test Evaluation

#### Overall Performance
- **Accuracy**: 93%
- **Precision**: 0.9318
- **Recall**: 0.9311
- **F1 Score**: 0.9313
- **AUC**: 0.9939

---

### Class-wise Metrics

| Class      | Precision | Recall | F1-Score | Support |
|------------|-----------|--------|----------|---------|
| **Cardboard** | 1.00      | 0.95   | 0.97     | 81      |
| **Glass**     | 0.92      | 0.92   | 0.92     | 101     |
| **Metal**     | 0.93      | 0.93   | 0.93     | 82      |
| **Paper**     | 0.96      | 0.97   | 0.97     | 119     |
| **Plastic**   | 0.89      | 0.90   | 0.89     | 97      |
| **Trash**     | 0.83      | 0.86   | 0.84     | 28      |

---

### Key Insights
1. **Strengths**:
   - The model performs exceptionally well across most categories, especially "Cardboard," "Glass," "Metal," and "Paper."
   - High AUC (0.9939) demonstrates strong class separation.
   - Decently high accuracy of ~93%

2. **Weaknesses**:
   - Lower performance on "Trash" (F1-score: 0.84), likely due to limited examples and variability.
   - Slightly lower F1-score for "Plastic" (0.89) indicates room for improvement.

---


In [11]:
# Evaluate on the test set
model = efficientnet_b0(num_classes=6)  # Adjust num_classes as per your task
model.load_state_dict(torch.load('Model_save/Efficientnet_trash_classifier_final.pth'))  # Load the weights
model.to(device)  # Move to the appropriate device (CPU/GPU)
model.eval()  # Set to evaluation mode
validate_model_with_metrics_and_report(model, test_loader)

/tmp/ipykernel_162476/78138125.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('Model_save/Efficientnet_trash_classifier_final.pth'))  #


Classification Report:
              precision    recall  f1-score   support

   cardboard       1.00      0.95      0.97        81
       glass       0.92      0.92      0.92       101
       metal       0.93      0.93      0.93        82
       paper       0.96      0.97      0.97       119
     plastic       0.89      0.90      0.89        97
       trash       0.83      0.86      0.84        28

    accuracy                           0.93       508
   macro avg       0.92      0.92      0.92       508
weighted avg       0.93      0.93      0.93       508

Precision: 0.9318
Recall: 0.9311
F1 Score: 0.9313
AUC: 0.9939


(np.float64(0.9318254002345704),
 np.float64(0.9311023622047244),
 np.float64(0.9313297124885986),
 np.float64(0.9939290936080889),
 '              precision    recall  f1-score   support\n\n   cardboard       1.00      0.95      0.97        81\n       glass       0.92      0.92      0.92       101\n       metal       0.93      0.93      0.93        82\n       paper       0.96      0.97      0.97       119\n     plastic       0.89      0.90      0.89        97\n       trash       0.83      0.86      0.84        28\n\n    accuracy                           0.93       508\n   macro avg       0.92      0.92      0.92       508\nweighted avg       0.93      0.93      0.93       508\n')

[codecarbon INFO @ 00:07:12] Energy consumed for RAM : 0.000186 kWh. RAM Power : 5.572976589202881 W
[codecarbon INFO @ 00:07:12] Energy consumed for all CPUs : 0.001417 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 00:07:12] 0.001602 kWh of electricity used since the beginning.
[codecarbon INFO @ 00:07:12] 0.004928 g.CO2eq/s mean an estimation of 155.4245696592646 kg.CO2eq/year
[codecarbon INFO @ 00:07:27] Energy consumed for RAM : 0.000209 kWh. RAM Power : 5.572976589202881 W
[codecarbon INFO @ 00:07:27] Energy consumed for all CPUs : 0.001594 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 00:07:27] 0.001803 kWh of electricity used since the beginning.
[codecarbon INFO @ 00:07:42] Energy consumed for RAM : 0.000232 kWh. RAM Power : 5.572976589202881 W
[codecarbon INFO @ 00:07:42] Energy consumed for all CPUs : 0.001771 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 00:07:42] 0.002003 kWh of electricity used since the beginning.
[codecarbon INFO @ 00:07:57] Energy consumed for 

#### Step 9: Finish Code-Carbon Tracking

In [12]:
emissions = tracker.stop()
emissions_in_grams = emissions * 1000  # Convert kg to grams
print(f"Total CO2 emissions: {emissions_in_grams:.2f} g")

[codecarbon INFO @ 00:08:15] Energy consumed for RAM : 0.000282 kWh. RAM Power : 5.572976589202881 W
[codecarbon INFO @ 00:08:15] Energy consumed for all CPUs : 0.002151 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 00:08:15] 0.002433 kWh of electricity used since the beginning.


Total CO2 emissions: 0.90 g


/home/platinumfish/miniconda3/envs/Climate_CNN_3.10/lib/python3.10/site-packages/codecarbon/output_methods/file.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(total.values)])])
